In [75]:
# install main library YFinance
!pip install yfinance

In [76]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

import time
from datetime import date


# Question 1 Average growth of GDP in 2023

In [77]:
start_1 = date(year=2022, month=1, day=1)

# Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1)
gdpc1 = pdr.DataReader("GDPC1", "fred", start=start_1)

# Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago)
gdpc1['gdpc1_us_yoy'] = gdpc1.GDPC1/gdpc1.GDPC1.shift(4)-1

# Find the average YoY growth in 2023 (average from 4 YoY numbers). Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer 5.7
gdpc1.gdpc1_us_yoy.mean().round(3)

0.025

# Question 2. Inverse "Treasury Yield"

In [78]:
start_2 = date(year=2000, month=1, day=1)

# Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2, https://fred.stlouisfed.org/series/DGS10)
dgs2 = pdr.DataReader("DGS2", "fred", start=start_2)
dgs10 = pdr.DataReader("DGS10", "fred", start=start_2)

# Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join())
dgs2 = dgs2.join(dgs10)

# Calculate the difference dgs10-dgs2 daily.
dgs2['diff_DGS10_DGS2'] = dgs2.DGS10 - dgs2.DGS2

# Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.
dgs2.diff_DGS10_DGS2.min().round(1)

-1.1

# Question 3. Which Index is better recently?

In [79]:
# Download two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/)
sp500_daily = yf.download(tickers = "^GSPC",
                          start = "2019-04-09",
                          end = "2024-04-09",
                          interval = "1d")
ipcm_daily = yf.download(tickers = "^MXX",
                         start = "2019-04-09",
                         end = "2024-04-09",
                         interval = "1d")

# Compare 5Y growth for both (between 2019-04-09 and 2024-04-09)
growth_sp500 = sp500_daily['Adj Close'][-1] / sp500_daily['Adj Close'][0] - 1
growth_ipcm = ipcm_daily['Adj Close'][-1] / ipcm_daily['Adj Close'][0] - 1

# Select the higher growing index and write down the growth in % (closest integer %).
max(growth_sp500.round(2) * 100, growth_ipcm.round(2) * 100)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


81.0

# Question 4. 52-weeks range ratio (2023) for the selected stocks

In [80]:
# Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.
stocks_daily_4 = yf.download(tickers = "2222.SR BRK-B AAPL MSFT GOOG JPM",
                           start = "2023-01-01",
                           end = "2023-12-31",
                           interval = "1d",
                           group_by = 'tickers')


# Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value. Round the result to two decimal places (e.g. 0.1575 will be 0.16)
SR_range_ratio = ((stocks_daily_4["2222.SR"]["Adj Close"].max() - stocks_daily_4["2222.SR"]["Adj Close"].min()) / stocks_daily_4["2222.SR"]["Adj Close"].max()).round(2)
BRKB_range_ratio = ((stocks_daily_4["BRK-B"]["Adj Close"].max() - stocks_daily_4["BRK-B"]["Adj Close"].min()) / stocks_daily_4["BRK-B"]["Adj Close"].max()).round(2)
AAPL_range_ratio = ((stocks_daily_4["AAPL"]["Adj Close"].max() - stocks_daily_4["AAPL"]["Adj Close"].min()) / stocks_daily_4["AAPL"]["Adj Close"].max()).round(2)
MSFT_range_ratio = ((stocks_daily_4["MSFT"]["Adj Close"].max() - stocks_daily_4["MSFT"]["Adj Close"].min()) / stocks_daily_4["MSFT"]["Adj Close"].max()).round(2)
GOOG_range_ratio = ((stocks_daily_4["GOOG"]["Adj Close"].max() - stocks_daily_4["GOOG"]["Adj Close"].min()) / stocks_daily_4["GOOG"]["Adj Close"].max()).round(2)
JPM_range_ratio = ((stocks_daily_4["JPM"]["Adj Close"].max() - stocks_daily_4["JPM"]["Adj Close"].min()) / stocks_daily_4["JPM"]["Adj Close"].max()).round(2)

max(SR_range_ratio, BRKB_range_ratio, AAPL_range_ratio, MSFT_range_ratio, GOOG_range_ratio, JPM_range_ratio)

[*********************100%%**********************]  6 of 6 completed


0.42

# Question 5. Dividend Yield

In [81]:
# Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023.
# Sum up all dividends paid in 2023 per company

tickers = yf.Tickers('2222.SR BRK-B AAPL MSFT GOOG JPM')

sum_div_2222_SR = sum(tickers.tickers['2222.SR'].dividends[(tickers.tickers['2222.SR'].dividends.index >='2023-01-01') & (tickers.tickers['2222.SR'].dividends.index <='2023-12-31')])
sum_div_BRK_B = sum(tickers.tickers['BRK-B'].dividends[(tickers.tickers['BRK-B'].dividends.index >='2023-01-01') & (tickers.tickers['BRK-B'].dividends.index <='2023-12-31')])
sum_div_AAPL = sum(tickers.tickers['AAPL'].dividends[(tickers.tickers['AAPL'].dividends.index >='2023-01-01') & (tickers.tickers['AAPL'].dividends.index <='2023-12-31')])
sum_div_MSFT = sum(tickers.tickers['MSFT'].dividends[(tickers.tickers['MSFT'].dividends.index >='2023-01-01') & (tickers.tickers['MSFT'].dividends.index <='2023-12-31')])
sum_div_GOOG = sum(tickers.tickers['GOOG'].dividends[(tickers.tickers['GOOG'].dividends.index >='2023-01-01') & (tickers.tickers['GOOG'].dividends.index <='2023-12-31')])
sum_div_JPM = sum(tickers.tickers['JPM'].dividends[(tickers.tickers['JPM'].dividends.index >='2023-01-01') & (tickers.tickers['JPM'].dividends.index <='2023-12-31')])

# Divide each value by the closing price (Adj.Close) at the last trading day of the year.

div_yield_2222_SR = sum_div_2222_SR * 100 / stocks_daily_4["2222.SR"]["Adj Close"][-2]
div_yield_BRK_B = sum_div_BRK_B * 100 / stocks_daily_4["BRK-B"]["Adj Close"][-1]
div_yield_AAPL = sum_div_AAPL * 100 / stocks_daily_4["AAPL"]["Adj Close"][-1]
div_yield_MSFT = sum_div_MSFT * 100 / stocks_daily_4["MSFT"]["Adj Close"][-1]
div_yield_GOOG = sum_div_GOOG * 100 / stocks_daily_4["GOOG"]["Adj Close"][-1]
div_yield_JPM = sum_div_JPM * 100 / stocks_daily_4["JPM"]["Adj Close"][-1]

# Find the maximum value in % and round to 1 digit after the decimal point

max(div_yield_2222_SR, div_yield_BRK_B, div_yield_AAPL, div_yield_MSFT, div_yield_GOOG, div_yield_JPM).round(1)

2.8